# Описание проекта


Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Задача исследования**: Проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Исследование базы данных

In [2]:
query = '''SELECT * 
           FROM books 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''SELECT * 
           FROM authors 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''SELECT * 
           FROM publishers 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''SELECT * 
           FROM ratings 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''SELECT * 
           FROM reviews 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Выгрузка работает правильно, с данными все нормально, поэтому приступим к решению задач

# Решение заданий

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''SELECT COUNT(*)
           FROM books
           WHERE publication_date > '2000-01-01'
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
query =''' WITH rate AS (
               SELECT book_id,
                      AVG(rating) AS avg_rate
               FROM ratings
               GROUP BY book_id 
           ),
           review AS(
               SELECT book_id,
                      COUNT(review_id) AS count_review
               FROM reviews
               GROUP BY book_id
           )
           SELECT books.book_id, 
                  books.title, 
                  review.count_review, 
                  rate.avg_rate
           FROM books 
           LEFT JOIN review ON review.book_id = books.book_id
           LEFT JOIN rate ON books.book_id = rate.book_id
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,title,count_review,avg_rate
0,652,The Body in the Library (Miss Marple #3),2.0,4.500000
1,273,Galápagos,2.0,4.500000
2,51,A Tree Grows in Brooklyn,5.0,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,839,The Prophet,4.0,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,4.0,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,148,Christine,3.0,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


In [9]:
query = '''SELECT books.book_id,
                  books.title,
                  review.count_review, 
                  rate.avg_rate
           FROM books 
           LEFT JOIN(
                    SELECT book_id,
                           COUNT(review_id) AS count_review
                    FROM reviews
                    GROUP BY book_id 
           ) AS review ON books.book_id = review.book_id
           LEFT JOIN(
                    SELECT book_id,
                           AVG(rating) AS avg_rate
                    FROM ratings
                    GROUP BY book_id 
           ) AS rate ON books.book_id = rate.book_id
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count_review,avg_rate
0,652,The Body in the Library (Miss Marple #3),2.0,4.500000
1,273,Galápagos,2.0,4.500000
2,51,A Tree Grows in Brooklyn,5.0,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,839,The Prophet,4.0,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,4.0,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,148,Christine,3.0,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


В последних двух столбцах посчитаны количество обзоров и средняя оценка к каждой книге

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = '''SELECT publishers.publisher_id, 
                  publishers.publisher, 
                  COUNT(books.book_id) as count_books
           FROM publishers 
           JOIN books ON publishers.publisher_id = books.publisher_id
           WHERE books.num_pages > 50
           GROUP BY publishers.publisher_id
           ORDER BY count_books DESC
           LIMIT 1
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


Издательство Penguin Books выпустило 42 книги.

### Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query = '''WITH ratedbooks AS(
           SELECT books.author_id, 
                  books.book_id,
                  COUNT(ratings.rating_id) as count,
                  AVG(ratings.rating) as avgrating
           FROM books 
           JOIN ratings ON books.book_id = ratings.book_id
           GROUP BY books.book_id
           HAVING COUNT(ratings.rating_id) > 50)
           
           SELECT authors.author,
                  AVG(ratedbooks.avgrating) as author_rating
           FROM authors 
           JOIN ratedbooks ON authors.author_id = ratedbooks.author_id
           GROUP BY authors.author
           ORDER BY author_rating DESC
           LIMIT 1
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,author_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средний оценкой книг - это Джоан Роулинг

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''WITH num_ratings AS (
           SELECT ratings.username as username,
                  COUNT(ratings.rating)
           FROM ratings
           GROUP BY ratings.username
           HAVING COUNT(ratings.rating) > 48),
        
           num_reviews AS (
           SELECT num_ratings.username as username,
                  COUNT(reviews.text)
           FROM num_ratings JOIN reviews
           ON num_ratings.username = reviews.username
           GROUP BY num_ratings.username)
           
           SELECT AVG(num_reviews.count)
           FROM num_reviews
           
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24

# Вывод

В данном проекте, мы подключились к серверу с базой данных, и ответили на следующие вопросы с помощью SQL:

1) Посчитайте, сколько книг вышло после 1 января 2000 года. **ОТВЕТ: 819**
   
2) Для каждой книги посчитайте количество обзоров и среднюю оценку. **ОТВЕТ: Выгружен в виде таблицы**

3) Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры. **ОТВЕТ: Издательство Penguin Books выпустило 42 книги.**

4) Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками. **ОТВЕТ: Автор с самой высокой средний оценкой книг - это Джоан Роулинг**

5) Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок. **ОТВЕТ: Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24**